In [ ]:
import os
import json
import pandas as pd

# Useful functions - Convert json files into single csv
def convert_json_to_df(data_file):
    with open(data_file, "r") as read_file:
        data = json.load(read_file)

    data = pd.json_normalize(data['data'])
    return data

def get_files_in_folders(base_path, types):
    all_file_paths = []
    for i in types:
        files = os.listdir(base_path + i)
        full_path_files = [base_path + i + '/' + file for file in files]
        all_file_paths.extend(full_path_files)
    return all_file_paths

def extract_specific_attribute(full_file_paths, attributes):
    list_of_dataframes = []
    for data_file in full_file_paths:
        df = convert_json_to_df(data_file)
        columns_to_extract = df.columns[df.columns.isin(attributes)] # Only filter attribute if present
        list_of_dataframes.append(df[columns_to_extract]) 
    return combine_and_tidy_dfs(list_of_dataframes)


def combine_and_tidy_dfs(list_of_dataframes, indexer='attributes.created_at'):
    df = pd.concat(list_of_dataframes)
    if df.columns[df.columns.isin([indexer])].any():
        df[indexer] = pd.to_datetime(df[indexer])
        df = df.set_index(indexer)
        df = df.sort_index()
    return df